# Side Channel Analysis Metric API
There exists very few comprehensive open-source libraries for side-channel analysis. Libraries that do exist are difficult to use and lack the proper documentation. This Jupyter Notebook outlines some of the most useful metrics when conducting side channel analysis in an easy to understand medium. These metrics do not perform an attack, rather they are used to assess gain insight into the cryptographic system. Many of the metrics in this API require data to be pre-formatted prior to use. However, each metric explains what programmer needs to provide. 

In [9]:
import numpy as np
from scipy import stats
import math
import matplotlib.pyplot as plt



ValueError: negative dimensions are not allowed

## Signal-to-Noise Ratio 
The signal-to-noise ratio of a signal is defined as the ratio of a signal's data component to the signal's noise component. For side-channel analysis, the SNR of a power trace relates to the ability for an attacker to obtain information from a power trace during an attack. The effectiveness of side channel attack increases for larger SNR values since the signal leakage is more prominent relative to the noise of the signal. Typically recorded power traces need to be partitioned into different sets called labels. 

$$
SNR = \frac{VAR(L_d)}{VAR(L_n)} = \frac{\sum_{v=0}^{V} (\hat{\mu_v}^2 - \hat{\mu})^2}{\hat{\sigma}^2}
$$
The resulting array is the value of the SNR at a given discrete time sample. Windows of the resulting trace where the magnitude of the SNR is high may also indicate an area of interest since it implies that there exists a significant amount of leakage at that sample.

In [8]:
# Signal to Noise Ratio metric 
#   - labels: An array of arrays. Each index of the labels array (i.e. labels[0])
#     is a label group containing the corresponding power traces.
# return: the SNR signal 
def signal_to_noise_ratio(labels):
   # statistical mean and variances of each set 
   set_means = []
   set_variances = []
   
   for label in labels:
       set_means.append(np.mean(label, axis=0)) # take the mean along the column
       set_variances.append(np.var(label, axis=0)) # take the variance along the column
   
   # calculate overall mean and variance
   overall_mean= np.mean(set_means, axis=0)              
   overall_variance = np.var(set_variances, axis=0)

   # perform SNR calculation
   l_d = np.zeros(len(set_means[0]))
   for mean in set_means:
       l_d = np.add(l_d, np.square(np.subtract(mean, overall_mean)))
   l_n = overall_variance

   snr = np.divide(l_d, l_n)

   return snr

## Score and Rank
The score and rank metric is a helpful metric to use both during an attack and in the analysis of a system. This metric relies primarily on two steps, first, the full-length cryptographic key is split into multiple segments, called partitions. Typically, these partitions are the size of a byte, but can be either larger or shorter depending on the particular encryption algorithm and user implementation. This means that for partitions that are the size of a byte, there are 256 key possibilities. Next, a scoring function needs to be specified. This function is arbitrary but needs to return a numerical scores such that the higher the score, the more likely a given input key, k, actually produced the traces. Using the scoring function, for each partition, each possible key is ranked from the highest score to the lowest score. The idea of ranking and scoring the key guesses is that as the number of traces increases, the rank will converge to the point that the actual key will remain in the 1st rank, or very close to it for all key partitions. 

In [9]:
# Score metric: Ranks each key guess in a key partition based on a scoring function
#   - traces: The trace set to be evaluated
#   - score_fcn: Function callback that takes two arguments, traces and a guess candidate
#                and returns a "score" such that the higher the value, the more likely the 
#                key candidate is the actual key
#   - partitions: The number of partitions of the key full key.
# return: A 2D array rank. The value rank[i] are the key guess rankings for partition i. 
#         The value of rank[i][0] is the highest ranked key guess for partition i.
def score_and_rank(traces, score_fcn, key_candidates, partitions):
        dtype = [('key', int), ('score', 'float64')]
        ranks = []
        # for each key partition        
        for i in range(partitions): 
            partition_scores = np.array([], dtype=dtype)
            
            # for each key guess in the partition score the value and add to list
            for k in key_candidates:
                score_k = score_fcn(traces, k)
                key_score = np.array([(k, score_k)], dtype=dtype)
                partition_scores = np.append(partition_scores, key_score)
                
            # rank each key where partition_ranks[0] is the key that scored the highest
            partition_ranks = np.array([key_score[0] for key_score in np.sort(partition_scores, order='score')[::-1]])
            
            ranks.append(partition_ranks)
        return ranks

## Success Rate
In the analysis of a system, the Success Rate metric can be used alongside the Score and Rank metrics in order to help determine the security of a system. This metric is typically conducted by an evaluator not an attacker since the correct key for the cryptographic system must be known. The success rate of a given experiment, i, is defined as 1 if the correct key is ranked within the top o key guesses. The order, o, can be any value between 1 and K where K is the number of key guesses. When o is 1, the success rate is only 1 if the correct key was ranked first out of all possible key guesses. Similarly, if o is 2, the success rate will be 1 if the correct key is ranking within the top 2 ranks. 
\begin{equation}
    SR_{o}^{i}=
    \begin{cases}
        \text{1 if } k_{c} \in [guess_{1}, guess_{2}, ..., guess_{o}]\\
        \text{0 otherwise}
    \end{cases}
\end{equation}
The overall success rate is defined as the sum of the success rates of all experiments divided by the number of experiments. Lower success rates indicate a high degree of system security and vise versa. 
\begin{equation}
    SR_{o}= \frac{1}{p}\sum_{i=1}^{p}SR_{o}^{i}
\end{equation}

## Guessing Entropy
Guessing Entropy is another similar metric that can analyze the security of a system. The Guessing Entropy is defined as the sum of the natural log of the rank of the correct key for all experiments. 
\begin{equation}
    GE^{i}=log_{2}(rank_{k_{c}})
\end{equation}
\begin{equation}
    GE = \frac{1}{p}\sum_{i=1}^{p}GE^{i}
\end{equation}
The guessing entropy metric conveys the average workload left in the attack. As the guessing entropy decreases the certainty of the key guess increases, to the point where at a GE of 0, the key guess is certain.

In [10]:
# Success Rate and Guessing Entropy Metric
#   - correct_key: the correct key of the cryptographic system, typically a key partition in practice
#   - ranks: The ranks of key guesses for a given experiment
#   - num_experiments: The number of experiments conducted
# return: The values of success_rate and guessing_entropy for the given number of experiments
def success_rate_guessing_entropy(correct_key, ranks, order, num_experiments):
    success_rate = 0
    guessing_entropy = 0
    
    # for each experiment
    for i in range(num_experiments):
        
        # check if correct key is within o ranks
        for j in range(order):
            if ranks[i][j] == correct_key:
                success_rate += 1
                break
        
        # guessing entropy is the log2 of the rank of the correct key
        guessing_entropy += math.log2(ranks[i].index(correct_key) + 1)
    
    success_rate = success_rate / num_experiments
    guessing_entropy = guessing_entropy / num_experiments
    
    return success_rate, guessing_entropy
    

## Pearson Correlation Coefficient
Correlation can be used as a metric that compares two different trace sets. The first set of power traces are observed leakages relating to an intermediate value V. The second set are predicted power traces that were derived using some sort of leakage model g(.) relating to an intermediate algorithm output value V for the correct key guess.
$$
p_{k_{c}} = \frac{\sum_{i=1}^{n} (l_{i}- \frac{1}{n} \sum_{i=1}^{n} l_{i}) (g(f(x_{i},k_{c})) - \frac{1}{n} \sum_{i=1}^{n} g(f(x_{i},k_{c})))}{{\sqrt{\sum_{i=1}^{n}({l_i - \frac{1}{n}\sum_{i=1}^n}{l_i})^2} \sqrt{\sum_{i=1}^n{(g(f(x_{i},k_{c}))-\frac{1}{n} \sum_{i=1}^{n}{g(f(x_{i},k_{c}))})^2}}}}
$$
If the absolute value of the correlation between predicted traces modeled using the correct key, is greater than that of other key guesses, then the key will likely be able to be derived from a side channel attack. 

In [25]:
# Pearson's Correlation Coefficient Metric
#   - predicted_leakage: predicted traces associated with intermediate values and a key guess and a plaintext value
#   - observed_leakage: actual power traces observed with a given plaintext
# returns: the correlation coefficient and p-value between each trace
# TODO: Write an implementation similar to the example in the hardware hacking handbook
def pearson_correlation(predicted_leakage, observed_leakage):
    correlation_coeff, p_value = stats.pearsonr(predicted_leakage, observed_leakage)
    return abs(correlation_coeff), p_value

## T-Test with TVLA

In [14]:
# T-test with TVlA metric:  In general |t| > th where th = 4.5 means that the system leaks information about the cryptographic key.
#   - fixed:  Trace set recorded with a fixed plaintext 
#   - random: Trace set recorded with a random set of plaintexts
# return: t_statistic and p-value
def t_test_tvla(fixed, random):
    
    # determine t_statistic and p-value using scipy 
    t_statistic, p_value = stats.ttest_ind(fixed, random)
    
    # high t-statistic and low p-values indicate that a given time sample leaks information
    return t_statistic, p_value